Generate local predictions of data located in edgebox and compare them with the previously online created results (Just generation. Not simulation)

 Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta

In [ ]:
import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

DB Config

In [ ]:
#####DB Config
host= "Devedgev32"
port=27017

Set Parameters

Funtion to remove t_bett from t_spindel and t_motor

Function to smooth a signal

Function to create a data set, where each data point is a window of n steps (look window feature)

Set the time period to get the data

In [ ]:
#start= datetime.strptime("2022-09-20T22:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")# Trocken iso
#end= datetime.strptime ("2022-09-21T08:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
#start= datetime.strptime("2023-02-02T10:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")# Trocken iso
#end= datetime.strptime ("2023-02-02T18:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-01-31T17:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") #Messtaster mit Fräsversuch
# end= datetime.strptime ("2023-01-31T23:59:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-02-23T18:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") #Messtaster mit Fräsversuch
# end= datetime.strptime ("2023-02-24T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
start= datetime.strptime("2023-04-16T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2023-04-19T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

Read data from the mongo_DB database and apply preprocessing

In [ ]:
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None#pd.DataFrame({})
signals= pd.DataFrame({})
List=[]
x=0
i=0
dicss={  "T_KWAV1": "/NCK/State/adbr[2400]",
      "T_KWAR1_MS": "/NCK/State/adbr[2404]",
      "T_KWR_MS": "/NCK/State/adbr[2408]",
      "V_KW_MS": "/NCK/State/adbr[2412]",
      "V_KWAV1": "/NCK/State/adbr[2416]",
      "V_KWAV2": "/NCK/State/adbr[2480]",
      "T_KSM1": "/NCK/State/adbr[2420]",
      "T_SCHLITTEN_Y": "/NCK/State/adbr[2424]",
      "T_SPI_MB1": "/NCK/State/adbr[2428]",
      "T_SPI_MB2": "/NCK/State/adbr[2436]",
      "T_RAUM": "/NCK/State/adbr[2440]",
      "T_BETT_X": "/NCK/State/adbr[2444]",
      "T_STAE_Y": "/NCK/State/adbr[2448]",
      "T_BETT_Z": "/NCK/State/adbr[2452]",
      "T_Spindel": "/NCK/State/adbr[2456]",
      "ActSpeed_S": "/Channel/Spindle/actSpeed[u1,1]",
      "Power_S": "/Channel/MachineAxis/aaPower[u1,8]",
      "Torque_S": "/Channel/MachineAxis/aaTorque[u1,8]",
      "Current_S": "/Channel/MachineAxis/aaCurr[u1,8]",
      "T_Motor_S": "/DriveVsa/Drive/r0035[u8]"}
Names=[  "T_BETT_X",
      "T_Spindel",
      "T_Motor_S"]
[Names.append(i) for i in dicss.keys()]
last_prediction= -500
prev_record_time= None
### Go through events in DB
# signals=None
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        if record['date']< start or record['date'] > end:
            continue
        
       
        keys= record['raw_data'].keys()
        record['raw_data']['date']=record['date'] + timedelta(hours= +2)
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
     
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals=lf_signal_point.copy()
        else :
            # pd.concat(signalslf_signal_point.copy(),)
                signals=signals.append(lf_signal_point,ignore_index=True)

        if len(List)==0:
            List.append(lf_signal_point.copy())
        else:
            List.append(lf_signal_point)#,ignore_index= True)#signals= 
signals=signals.append(List,ignore_index=True)
signals.reset_index(inplace=True)


In [ ]:
# signals=pd.read_csv('C:\\Users\\kwa001\\Downloads\\Flow_creator\\data.csv')

In [ ]:
import re
for col in signals.columns:
    if re.search(r"^[T_]",col):
        signals.loc[:,col]=signals.loc[:,col].round(2)

In [ ]:
signals.loc[: ,'V_KW_MS']

In [ ]:
# (T_Rück - T_Vor) * V [l/min] * 1,16 *60/1000 => Kühlleistung in kW
dff=(signals.loc[:,'T_KWR_MS'] - signals.loc[:,'T_KWAV1']) *signals.loc[: ,'V_KW_MS']*(1000/60) * (1.16) *(60/1000)
signals['Kühlleistung']=dff

# signals.insert(0,'Kühlleistung',dff)

In [ ]:
# import plotly.io as pio

columns=['T_BETT_X', 'T_Spindel', 'T_Motor_S', 'T_BETT_Z', 'Torque_S'#]#,
       ,'ActSpeed_S', 'T_STAE_Y', 'Power_S', 'T_KWR_MS', 'T_SCHLITTEN_Y'#]#,
       ,'T_RAUM', 'V_KW_MS', 'Current_S', 'T_KWAR1_MS', 'T_KWAV1', 'V_KWAV1']
    #    'T_KSM1', 'V_KWAV2', 'T_SPI_MB1', 'T_SPI_MB2', 'T_Motor_Z',
    #    'T_Motor_X', 'Power_B', 'Power_Y', 'Torque_Z', 'Current_A', 'T_Motor_A',
    #    'Torque_Y', 'Current_B', 'Power_A', 'Power_X', 'T_Motor_B', 'T_Motor_Y',
    #    'Power_Z', 'Torque_A', 'Current_X', 'Current_Z', 'Torque_B', 'Torque_X',
    #    'Current_Y']
columnss=['T_BETT_X°', 'T_Spindel°', 'T_Motor_S°', 'T_BETT_Z°', 'Torque_S'#]#,
       ,'ActSpeed_S(R\M)', 'T_STAE_Y°', 'Power_S', 'T_KWR_MS°', 'T_SCHLITTEN_Y°','T_RAUM°', 'V_KW_MS', 'Current_S', 'T_KWAR1_MS°', 'T_KWAV1°', 'V_KWAV1','Kühlleistung']#,

columns1=['T_BETT_X(°C)', 'T_Spindel(°C)', 'T_Motor_S(°C)', 'T_BETT_Z(°C)', 'Torque_S(NM)'#]#,
       ,'ActSpeed_S(NM)', 'T_STAE_Y(°C)', 'Power_S(KW)', 'T_KWR_MS(°C)', 'T_SCHLITTEN_Y(°C)','T_RAUM(°C)', 'V_KW_MS(L/m)', 'Current_S', 'T_KWAR1_MS(°C)', 'T_KWAV1(°C)', 'V_KWAV1(L/m)','Kühlleistung(KW)']

scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'

fig= make_subplots(rows=len(columns),cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02,subplot_titles=columnss)
ii=1
for i in range(len(columns)):

    #fig= make_subplots(rows=9,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02,subplot_titles=['t_bett',' Hals','t_spindel','t_schlitten','Schlitten_copm','Gesamte_comp','Fehler_comp',' welle','Prediction_30S'])


    fig.add_trace(go.Scatter(x=signals['date'], y=signals[columns[i]], name=columnss[i], mode= scatter_mode), row= ii, col= 1)
    fig.update_yaxes(title_text= columns1[i], row= ii, col= 1)
    ii=ii+1







#,range=[0,len(df_list2[0])]
#fig.update_xaxes( title="seconds" )#range=[0,len(df_list2[0])] 


#pio.write_image(fig,'C:\\Users\\KWA001\\Downloads\\Neuer Ordner\\'+str(i)+'.png','png')
#fig.write_image('C:\\Users\\KWA001\\Downloads\\Neuer Ordner\\'+str(i)+'.png')

fig.update_layout(height=3000, width=1200, title_text='Machine HF5500' )#'Hals_comp_VS_Hals_PT!_Glied _Versuch{}'.format(j)
fig.show()



In [ ]:
signals.to_csv('G:\\Innovations@HELLER\\DN\KI\\flow_creator\\data.csv')